In [9]:
import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd
import json
import os
import os
import json
os.chdir('/home/aiops/zhuty/tinyllama/scripts/fewshot_eval')
from utils import remove_underline, extract_model_and_steps, CLASS_TO_TASKS_MAPPING, CLASS_TO_SEED_MAPPING, ALL_TASKS


In [11]:
BASE_PATH = '/home/aiops/zhuty/tinyllama/scripts/fewshot_eval/outputs/'
commands = []
data = []


high_priority=True
if high_priority:
    high_string = "-p high"
else:
    high_string = ""
# size="360M"
# iter_name = "iter-160000-ckpt-step-40000_hf"
#iter_name = "iter-110000-ckpt-step-27500_hf"
# iter_name = "iter-260000-ckpt-step-65000_hf"

size="1b"



iterations = []
short_model_names =  ['cc_merged_v2_8k' ,'cc_8k' , 'intramask_cc_8k', 'adamask_cc_merged_v2_8k', 'cc_merged_v1_8k', ]
model_names = ['cc_merged_v2_8k' ,'cc_8k' , 'intramask_cc_8k', 'adamask_cc_merged_v2_8k', 'cc_merged_v1_8k', 'cc_merged_v2_8k_intracccont']
# model_names = ['cc_merged_v2_8k_intracccont']

# for step in range(5000, 80000, 5000):
for step in range(70000, 80000, 5000):
    iter_name = f'iter-{step*8:06}-ckpt-step-{step}_hf'
    if step < 50000:
        # remove intracccont models
        iterations.append({"iter_name": iter_name, "model_names": short_model_names})
    else:
        iterations.append({"iter_name": iter_name, "model_names": model_names})

# model_names = ['BM25Chunk', 'IntraDoc', 'UniChunk', 'MixChunk']
# iterations = [({"iter_name": "iter-50000-ckpt-step-6250_hf", "model_names": model_names})]

selected_task = "obqa_rag1"
tasks = [task for task in ALL_TASKS if task["TASK_CLASS"] == selected_task]


# Loop through each set of iter_name and model_names
for iteration in iterations:
    iter_name = iteration["iter_name"]
    model_names = iteration["model_names"]

    print(f"Processing {iter_name} with models:")
    for model_name in model_names:
        print(f" - {model_name}")
    # Loop through each set of TASK_CLASS, shot_nums, and rag_num_docs_lst
    for task in tasks:
        TASK_CLASS = task["TASK_CLASS"]
        shot_nums = task["shot_nums"]
        rag_num_docs_lst = task["rag_num_docs_lst"]



        for task_name in  CLASS_TO_TASKS_MAPPING[TASK_CLASS]:
            # for shot_num in [1, 3, 5, 10, 20, 30, 48]:
            # for shot_num in [3,5,10,20,30,48]:
            # for shot_num in [24, 48]:
            for shot_num in shot_nums:
                for rag_num_docs in rag_num_docs_lst:
                    # if task_name == "yelp" and shot_num == 48:
                    #    continue
                    for model_name in model_names:
                    # for step in range(5000, 80000, 5000):
                    # for step in [75000]:
                        # iter_name = f'iter-{step*8:06}-ckpt-step-{step}_hf'
                        # model_name = 'cc_merged_v2_8k'
                        if '8k' not in model_name and 'cc' in model_name:
                            model_name = f"{model_name}_8k"
                        for seed_num in CLASS_TO_SEED_MAPPING[TASK_CLASS]:
                            full_name = f"tiny_LLaMA_{size}_8k_{model_name}-{iter_name}"
                            # full_name = "TinyLlama-1.1B-step-50K-105b"
                            # full_name = model_name
                            if TASK_CLASS == "obqa_rag":
                                result_path = os.path.join(BASE_PATH,full_name, f'{task_name}_{shot_num}_{seed_num}_{rag_num_docs}.json')
                            else:
                                result_path = os.path.join(BASE_PATH,full_name, f'{task_name}_{shot_num}_{seed_num}.json')

                            if not os.path.exists(result_path):
                                # print(f"File {result_path} does not exist")
                                # model_path = f"/home/aiops/zhuty/tinyllama/out/tiny_LLaMA_{size}_8k_{model_name}/{iter_name}"
                                model_path = f"tyzhu/tiny_LLaMA_{size}_8k_{model_name}_{iter_name}"
                                if model_name in ['BM25Chunk', 'IntraDoc', 'UniChunk', 'MixChunk'] :
                                    model_path = f"yuzhaouoe/{model_name}"


                                # check if config file exists
                                assert os.path.exists(f"{model_path}/config.json") or 'tyzhu/' in model_path or 'yuzhao' in model_path, "Missing config file for {}, convert it with \n {}".format(model_path, f"bash convert_to_hf_general.sh /home/aiops/zhuty/tinyllama/out/tiny_LLaMA_{size}_8k_{model_name} {iter_name.strip('_hf')}")

                                if "obqa_rag" in TASK_CLASS:
                                    assert str(rag_num_docs) != ""
                                else:
                                    assert str(rag_num_docs) == ""
                                jobname = f"eval{size.lower()}{remove_underline(task_name)}{remove_underline(model_name)}{seed_num}"[:40]
                                # curr = f""" sailctl job create {jobname} -g 1  --debug --image asia-docker.pkg.dev/sail-tpu-02/git-insea-io/common/image_registry/liuqian/tinyllama:v9   --command-line-args  ' bash /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/scripts/fewshot_eval/ ; bash single_task.sh {model_path} {task_name} {seed_num} {shot_num} {rag_num_docs}' {high_string} """
                                curr = (jobname, model_path, task_name, f"bash single_task.sh {model_path} {task_name} {seed_num} {shot_num} {rag_num_docs} ")

                                commands.append(curr)

                                continue
                            else:
                                # print("Existing", result_path)
                                result = json.load(open(result_path, 'r'))
                                if TASK_CLASS == "obqa_rag":
                                    result['num_docs'] = rag_num_docs
                                data.append(result)



Processing iter-560000-ckpt-step-70000_hf with models:
 - cc_merged_v2_8k
 - cc_8k
 - intramask_cc_8k
 - adamask_cc_merged_v2_8k
 - cc_merged_v1_8k
 - cc_merged_v2_8k_intracccont
Processing iter-600000-ckpt-step-75000_hf with models:
 - cc_merged_v2_8k
 - cc_8k
 - intramask_cc_8k
 - adamask_cc_merged_v2_8k
 - cc_merged_v1_8k
 - cc_merged_v2_8k_intracccont


In [12]:
assert len(commands) == 0 , "missing commands {}".format( len(commands))


AssertionError: missing commands 840

In [1]:
model_bash_commands = {}
long_task_commands = []
SHORT_TASKS = ['tweet_hate', 'tweet_offensive', 'sst2', 'memtrap', 'amazon', 'sst2']
high_string = "-p low"
for job_name, model_path, task_name,  bash_command in commands:
    job_name = job_name.replace("_", "").replace("-", "").lower()[:39]
    if task_name not in SHORT_TASKS:
        curr = f"""sailctl job create {job_name} -g 1  --debug --image asia-docker.pkg.dev/sail-tpu-02/git-insea-io/common/image_registry/liuqian/tinyllama:v9   --command-line-args  ' bash /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/scripts/fewshot_eval/ ; {bash_command} ' {high_string} """
        long_task_commands.append(curr)
        continue
    if (model_path, task_name) not in model_bash_commands:
        model_bash_commands[(model_path, task_name)] = []
    model_bash_commands[(model_path, task_name)].append(bash_command)
print("Long task commands: ", len(long_task_commands))


NameError: name 'commands' is not defined

In [14]:
print(len(long_task_commands))

840


In [15]:
print(" ;".join(long_task_commands))

sailctl job create eval1bnqobqaccmergedv28k42 -g 1  --debug --image asia-docker.pkg.dev/sail-tpu-02/git-insea-io/common/image_registry/liuqian/tinyllama:v9   --command-line-args  ' bash /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/scripts/fewshot_eval/ ; bash single_task.sh tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-560000-ckpt-step-70000_hf nq_obqa 42 2 1  ' -p low  ;sailctl job create eval1bnqobqaccmergedv28k43 -g 1  --debug --image asia-docker.pkg.dev/sail-tpu-02/git-insea-io/common/image_registry/liuqian/tinyllama:v9   --command-line-args  ' bash /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/scripts/fewshot_eval/ ; bash single_task.sh tyzhu/tiny_LLaMA_1b_8k_cc_merged_v2_8k_iter-560000-ckpt-step-70000_hf nq_obqa 43 2 1  ' -p low  ;sailctl job create eval1bnqobqaccmergedv28k44 -g 1  --debug --image asia-docker.pkg.dev/sail-tpu-02/git-insea-io/common/image_registry/liuqian/tinyllama:v9   --command-line-args  ' bash /home/aiops/zhuty/start.sh ; cd /home/ai

In [16]:
batch_size = 50
batches = [long_task_commands[i:i + batch_size] for i in range(0, len(long_task_commands), batch_size)]

with open(f"/home/aiops/zhuty/tinyllama/scripts/fewshot_eval/scripts/run_1b_{selected_task}_batches.sh", "w") as f:
    for batch in batches:
        for command in batch:
            f.write(f"{command} &\n")
        f.write("wait\n")
        f.write("sleep 1\n")  # Add a sleep between batches if needed


In [8]:
short_task_commands = []
high_string = "-p low"
for (model_path, task_name), bash_commands in model_bash_commands.items():
    # model_short_name = model_path.split('/')[-2].split('_8k')[1]
    model_short_name = model_path.split('/')[-1]
    jobname = f"{task_name}{model_short_name}".replace("_", "").replace("-", "").lower()[:39]
    curr = f"""sailctl job create {jobname} -g 1  --debug --image asia-docker.pkg.dev/sail-tpu-02/git-insea-io/common/image_registry/liuqian/tinyllama:v9   --command-line-args  ' bash /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/scripts/fewshot_eval/ ; {" ; sleep 2; ".join(bash_commands)}' {high_string}"""
    short_task_commands.append(curr)

In [14]:
len(short_task_commands)

In [15]:
print(" ; sleep 1; ".join([x for x in short_task_commands if 'g' in x]))

1

In [16]:
df = pd.DataFrame(data)

# ensure that no score is zero
assert len(df[df['score'] == 0]) == 0

sailctl job create sst2mixchunks5 -g 1  --debug --image asia-docker.pkg.dev/sail-tpu-02/git-insea-io/common/image_registry/liuqian/tinyllama:v9   --command-line-args  ' bash /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/scripts/fewshot_eval/ ; bash single_task.sh yuzhaouoe/MixChunk sst2 47 5  ' -p low


In [ ]:
def extract_step(x):
    # strip the _hf
    x = x[:-3]
    return int(x.split('-')[-1])

def extract_model_name(x):
    """
    Sample input: /home/aiops/zhuty/tinyllama/out/tiny_LLaMA_1b_8k_cc_8k/iter-200000-ckpt-step-25000_hf
    Sample output: tiny_LLaMA_1b_8k
    Args:
        x:
    Returns:

    """
    if 'TinyLlama/TinyLlama-1.1B-step-50K-105b' in x:
        return 'tinyllama-1.1b-50k-105b'
    if '/home/aiops/zhuty' in x:
        folder_name = x.split('/')[-2] # tiny_LLaMA_1b_8k_cc_8k
        return ("_").join(folder_name.split('_')[:4]) # tiny_LLaMA_1b_8k
    else:
        # sample model name: tyzhu/tiny_LLaMA_120M_8k_cc_8k_iter-240000-ckpt-step-60000_hf
        return x.strip("tyzhu/").split("_cc")[0]

def extract_pretrain_dataset(x):
    """
    Sample input: /home/aiops/zhuty/tinyllama/out/tiny_LLaMA_1b_8k_cc_8k/iter-200000-ckpt-step-25000_hf
    Sample output: cc_8k
    Args:
        x:
    Returns:

    """
    if '/home/aiops/zhuty' in x:
        folder_name = x.split('/')[-2] # tiny_LLaMA_1b_8k_cc_8k
        return ("_").join(folder_name.split('_')[4:]) # cc_8k
    else:
        # sample model name: tyzhu/tiny_LLaMA_120M_8k_cc_8k_iter-240000-ckpt-step-60000_hf
        return x.strip("tyzhu/").split("_8k_")[1]


In [ ]:
df

In [ ]:
short_task_commands = []
raw_commands = []
for (model_path, task_name), bash_commands in model_bash_commands.items():
    # model_short_name = model_path.split('/')[-2].split('_8k')[1]
    model_short_name = model_path.split('/')[-1]
    jobname = f"{task_name}{model_short_name}".replace("_", "").replace("-", "").lower()[:39]
    curr = f"""sailctl job create {jobname} -g 1  --debug --image asia-docker.pkg.dev/sail-tpu-02/git-insea-io/common/image_registry/liuqian/tinyllama:v9   --command-line-args  ' bash /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/scripts/fewshot_eval/ ; {" ; sleep 2; ".join(bash_commands)}' {high_string}"""
    short_task_commands.append(curr)
    raw_commands.extend(bash_commands)
print(" ;".join(raw_commands))

In [ ]:
import re

# Function to extract model name from the path
def extract_model_name(model_path):
    # Using regular expression to find model names like 'tiny_LLaMA_1b_cc_merged_8k' from the path
    match = re.search(r"tiny_LLaMA_(\w+)", model_path)
    return match.group(1) if match else "Unknown"

# Apply the function to extract model names
df['model_name'] = df['model'].apply(extract_model_name)

# Group the data by task, n_shot, and model and calculate the mean and std deviation
grouped_data = df.groupby(['task', 'n_shot', 'model_name'])['score'].agg(['mean', 'std']).reset_index()

# Display the processed data
grouped_data


unique_tasks = grouped_data['task'].unique()
names_to_meaning = {"8k_cc_8k": "random", "8k_cc_merged_v1_8k": "dense",
                    "8k_cc_merged_v2_8k": "bm25", "8k_cc_merged_v3_8k": "gen+bm25",
                    "8k_intramask_cc_8k": "intradoc"}
# Correcting the plot to ensure error bars for standard deviation are visible
def plot_task_data_corrected(task):
    plt.figure(figsize=(12, 6))
    task_data = grouped_data[grouped_data['task'] == task]

    # Plot each model separately to ensure error bars are added correctly
    for model in task_data['model_name'].unique():
        model_data = task_data[task_data['model_name'] == model]
        # no errorbars
        plt.plot(model_data['n_shot'], model_data['mean'], '-o', label=names_to_meaning[model])
        # plt.errorbar(model_data['n_shot'], model_data['mean'], yerr=model_data['std'], fmt='-o', capsize=5, label=names_to_meaning[model])

    plt.title(f'Performance of Models on {task} Task')
    plt.xlabel('Number of Shots')
    plt.ylabel('Average Score')
    plt.legend(title='Model', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()

# Plot corrected charts for each task
for task in unique_tasks:
    plot_task_data_corrected(task)
    plt.show()


In [ ]:
df['step'] = df['model'].apply(extract_step)
df['model_name'] = df['model'].apply(extract_model_name)
df['pretrain_dataset'] = df['model'].apply(extract_pretrain_dataset)
# df['pretrain_dataset'] = 'tllama'
# only filter the nshot to be 24
df = df[df['n_shot'] == 48]

In [ ]:
# Assuming `df` is your DataFrame
# Step 1: Group and aggregate
grouped = df.groupby(['model_name', 'task']).agg({
    'score': ['mean', 'std'],
    'seed': lambda x: list(x)
}).reset_index()

# Step 2: Pivot the table
# The tricky part here is dealing with multi-level column names resulting from aggregation
grouped.columns = ['pretrain_dataset', 'task', 'score_mean', 'score_std', 'seeds']
pivot_df = grouped.pivot_table(index='pretrain_dataset',
                               columns='task',
                               values=['score_mean', 'score_std', 'seeds'],
                               aggfunc='first').reset_index()

# Step 3: Rename columns for clarity
pivot_df.columns = ['_'.join(col).strip() if col[1] else col[0] for col in pivot_df.columns.values]


In [ ]:
import pandas as pd

# Assuming `df` is your DataFrame
# Step 1: Group and aggregate
grouped = df.groupby(['model_name', 'task']).agg({
    'score': [('score_mean', 'mean'), ('score_std', 'std')],
    'seed': [('seeds', lambda x: list(x))]
}).reset_index()

# Flatten the multi-level column names
grouped.columns = ['pretrain_dataset', 'task', 'score_mean', 'score_std', 'seeds']

# Combine mean and std into a single string in the format mean(std)
grouped['score_combined'] = grouped['score_mean'].round(2).astype(str) + "(" + grouped['score_std'].round(2).astype(str) + ")"

# Calculate average performance across all tasks for each pretrain_dataset
average_performance = grouped.groupby('pretrain_dataset')['score_mean'].mean().reset_index()
average_performance.columns = ['pretrain_dataset', 'average_performance']

# Step 2: Pivot the table
pivot_df = grouped.pivot_table(index='pretrain_dataset',
                               columns='task',
                               values=['score_combined', 'seeds'],
                               aggfunc='first').reset_index()

# Step 3: Rename columns for clarity
pivot_df.columns = ['_'.join(col).strip() if col[1] else col[0] for col in pivot_df.columns.values]

# Merge the average performance data
pivot_df = pivot_df.merge(average_performance, on='pretrain_dataset', how='left')

# Now, pivot_df is the desired DataFrame with an 'average_performance' column


In [ ]:
average_performance

In [ ]:

# sanity check, ensure that all seed columns contain the same list
for col in pivot_df.columns:
    if 'seeds' in col:
        for row in pivot_df[col]:
            try:
                seed_list = list((class_to_seed_mapping[TASK_CLASS]))
                assert row== list((class_to_seed_mapping[TASK_CLASS]))
            except:
                print(col)
                print(row)
                print(seed_list)


In [ ]:
cleaned_pivot_df = pivot_df.drop(columns=[col for col in pivot_df.columns if 'seeds' in col])
# rename the columns to remove the 'score_combined' prefix
cleaned_pivot_df.columns = [col.replace('score_combined_', '') for col in cleaned_pivot_df.columns]

In [ ]:
cleaned_pivot_df

In [ ]:
# random baselines:
# agnews: 25% ,
# amazon: 50%
# dbpedia: 7% ?
# sst2: 50%
# tweet hate: 50%
# tweet_offensive: 50%
# yelp: 50%

In [ ]:
pivot_df.columns

In [ ]:
df.columns

In [ ]:
# get all tasks
df['task'].unique()

In [ ]:
# plot the performance of 1b_8k on nq task
import matplotlib.pyplot as plt
INTERSTED_MODEL_NAME = 'tiny_LLaMA_1b_8k'
TASK_NAME = 'amazon'
df_1b_8k = df[df['model_name'] == INTERSTED_MODEL_NAME]
df_1b_8k = df_1b_8k[df_1b_8k['task'] == TASK_NAME]
# drop the columns that cannot be averaged
df_1b_8k = df_1b_8k.drop(columns=['model', 'task', 'model_name'])
# get the average over seeds, and standard deviation
df_1b_8k = df_1b_8k.groupby(['pretrain_dataset', 'step']).agg(['mean', 'std']).reset_index()
for dataset in df_1b_8k['pretrain_dataset'].unique():
    df_1b_8k_dataset = df_1b_8k[df_1b_8k['pretrain_dataset'] == dataset]
    plt.errorbar(df_1b_8k_dataset['step'], df_1b_8k_dataset['score']['mean'], yerr=df_1b_8k_dataset['score']['std'], label=dataset)
plt.legend()

### New version: based on the inputs itself

In [ ]:
print(len(commands))

In [ ]:
print(" ;".join(commands))

In [ ]:
d = eval_callables = {
    "nq": 1,
    "tq": 1,
    "wq": 1,
    "sst2": 1,
    "agnews": 1,
    "nq_obqa": 1,
    "hotpotqa": 1,
    "amazon": 1,
    "dbpedia": 1,
    "yelp": 1,
    "tweet_hate": 1,
    "tweet_offensive": 1,
    "squad": 1,
    "tq_obqa": 1,
}


In [ ]:
d.keys()

In [ ]:
print(commands[0])

In [ ]:
# read the output logs file
with open('/home/aiops/zhuty/tinyllama/scripts/fewshot_eval/outputs/logs', 'r') as file:
    data = file.readlines()
data = [ json.loads(x) for x in data]
print(data)

#### Misc utils files

In [ ]:
# copy the files
for ds in ['cc', 'cc_merged_v1', 'cc_merged_v2', 'intramask_cc']:
    print(f"mkdir -p tiny_LLaMA_1b_8k_{ds}_8k")
    print(
        f"rsync -ar --progress /s3/tinyllama/out_apr24/out/tiny_LLaMA_1b_8k_{ds}_8k/iter-380000-ckpt-step-47500_hf tiny_LLaMA_1b_8k_{ds}_8k/iter-380000-ckpt-step-47500_hf")


In [ ]:
size = "1b"
# iter_name = "iter-480000-ckpt-step-60000_hf"
# iter_name = "iter-380000-ckpt-step-47500_hf"
iter_name = "iter-480000-ckpt-step-60000_hf"

model_names = ['cc_merged_v2_8k', 'intramask_cc_8k', 'cc_merged_v2_8k_intracccont', 'cc_8k', 'adamask_cc_merged_v2_8k',
               'intramask_cc_merged_v2_8k']
for model_name in model_names:
    # print(f"python upload_to_hf.py /home/aiops/zhuty/tinyllama/out/tiny_LLaMA_{size}_8k_{model_name}/{iter_name}")
    assert os.path.exists(f"/home/aiops/zhuty/tinyllama/out/tiny_LLaMA_{size}_8k_{model_name}/{iter_name}/config.json")

In [ ]:
raw_commands = []
for model_name in ['BM25Chunk', 'IntraDoc', 'UniChunk', 'MixChunk']:
    raw_commands.append(f'bash single_task.sh yuzhaouoe/{model_name} memtrap 43 0 ')
print(" ;".join(raw_commands))